In [84]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import *
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer , KNNImputer
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.preprocessing import *
from sklearn.pipeline import *
from sklearn.tree import *
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import *

In [85]:
df = pd.read_csv('tabular-playground/train.csv')
df = df.drop('id' , axis = 1)

In [86]:
train_df = df.iloc[: ,  :-1]
label_df = df.iloc[: , -1]
features = train_df.columns

In [87]:
categorical_columns = []
numerical_columns = []
potential_categorical_columns = []

numeric_columns = train_df.select_dtypes(exclude=['object']).columns
for name in numeric_columns:
    unique_vals = len(df[name].value_counts())
    if(unique_vals <= 10):
        potential_categorical_columns.append(name)
    numerical_columns.append(name)
    
    
non_numeric_columns = train_df.select_dtypes(include=['object']).columns
for name in non_numeric_columns:
    categorical_columns.append(name)

In [88]:
def data_preprocessing(new_df , numerical_col , categorical_col):
    nn = []
    cc = []
    for f in new_df.columns:
        if f in numerical_col:
            nn.append(f)
        else:
            cc.append(f)
    # print(nn)
    # print(cc)
    # random.shuffle(nn)
    # random.shuffle(cc)
    numerical_pipe = Pipeline([
        ('impute', SimpleImputer()),
        ('scale', MinMaxScaler()),
    ])
    categorical_pipe = Pipeline([
        ('encode', OrdinalEncoder())
    ])
    preprocessor = ColumnTransformer([
        ('numeric', numerical_pipe, nn),
        ('categorical', categorical_pipe, cc),
    ])    
    return preprocessor


def feature_engineering(df , num_col , cat_col , try_add_m3m5_missing , try_self_defined_columns , try_drop_attr2 , try_drop_attr3):
    df_modified = df.copy()
    if try_add_m3m5_missing == True:
        df_modified['m_3_missing'] = df_modified.measurement_3.isna()
        df_modified['m_5_missing'] = df_modified.measurement_5.isna()
        cat_col = cat_col + ["m_3_missing" , "m_5_missing"]
    if try_drop_attr2 == True :
        df_modified = df_modified.drop(["attribute_2"] , axis = 1)
        num_col = list(set(num_col) - set(["attribute_2"]))
    if try_drop_attr3 == True :
        df_modified = df_modified.drop(["attribute_3"] , axis = 1)
        num_col = list(set(num_col) - set(["attribute_3"]))
    if try_self_defined_columns == True and try_drop_attr2 == False and try_drop_attr3 == False:
        cat_col = cat_col + ["attribute_2" , "attribute_3"]
        num_col = list(set(num_col) - set(["attribute_2" , "attribute_3"])) 

    return df_modified , num_col , cat_col

In [89]:
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm

In [90]:
model = tf.keras.models.load_model('Neural Network model weight/NN.h5')
# print(model.coef_)
try_add_m3m5_missing = True 
try_self_defined_columns = False
try_drop_attr2 = True
try_drop_attr3 = True
test_df = pd.read_csv('tabular-playground/test.csv')
id_ = test_df['id']
test_df = test_df.drop(["id"] , axis = 1)
new_test_df , num_col , cat_col = feature_engineering(test_df , numerical_columns ,
categorical_columns , try_add_m3m5_missing , try_self_defined_columns , try_drop_attr2 , try_drop_attr3)
preprocessor = data_preprocessing(new_test_df , num_col , cat_col)
preprocessed_new_test = preprocessor.fit_transform(new_test_df)
prob = model.predict(preprocessed_new_test)
df_subb = pd.read_csv('tabular-playground/sample_submission.csv')
df_subb.failure = prob
df_subb.to_csv('NN_eval.csv', index = None)

650/650 [==============================] - 0s 432us/step
